# Lesson 3.0.

# Putting it all together: Hackathon!!


### Objective: Find capacitor settings that minimize total substation real power (measuring line `l115`)

Rules of Engagement: You will complete the following steps below in order in the code blocks provided. Each step should be commented with proper documentation of your code and from which lesson you pulled (such as "pulled code from L2.6, S4.4 - Obtain measurement mRIDs").

Cheat Sheet: GridAPPS-D Sample App - opens and closes capacitors every 5 messages: https://github.com/GRIDAPPSD/gridappsd-sample-app/blob/master/sample_app/runsample.py


# 1. Import Libraries

In [1]:
import argparse
import json
import logging
import sys
import time
import pytz
import stomp

import numpy as np

In [17]:
from gridappsd import GridAPPSD, utils, DifferenceBuilder
from gridappsd import topics as t
from gridappsd.topics import simulation_input_topic, simulation_output_topic

# 2. Connect to GridAPPS-D Simulation

In [4]:
viz_simulation_id = "1768890986"

gapps = GridAPPSD(viz_simulation_id, "('localhost', 61613)", username='system', password='manager')

# 3. Obtain Equipment mRIDs

In [5]:
# model ID from run123demo.py
model_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"

In [6]:
topic = "goss.gridappsd.process.request.data.powergridmodel"

message = {
    "requestType": "QUERY_OBJECT_IDS",
    "resultFormat": "JSON",
    "modelId": model_mrid,
    "objectType": "LinearShuntCompensator"
} 

gapps.get_response(topic, message)
response_obj = gapps.get_response(topic, message)
ShuntComp_id = response_obj['data']['objectIds']
print(ShuntComp_id)

['_3DE55D2D-34D3-487E-9D6E-3A4DB1E38E47', '_939CA567-AA3D-4972-AABC-1D0AAF4859FE', '_CABBC3A1-66F5-4B9C-ACB9-476E2389D119', '_D6C44FF1-BC60-49D3-9438-DFAD1AED0A28']


In [7]:
topic = "goss.gridappsd.process.request.data.powergridmodel"
message = {
    "requestType": "QUERY_OBJECT_DICT",
    "resultFormat": "JSON",
    "modelId": model_mrid,
    "objectType": "ACLineSegment"
} 

 

gapps.get_response(topic, message)
response_obj = gapps.get_response(topic, message)
Line_info = response_obj['data']
Line_l115_mrid = [k['IdentifiedObject.mRID'] for k in Line_info if k['IdentifiedObject.name'] == 'l115'] 


# 4. Obtain Measurement mRIDs

In [38]:
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_MEASUREMENTS",
    "resultFormat": "JSON",
    "objectId": Line_l115_mrid[0]
}

response_obj = gapps.get_response(t.REQUEST_POWERGRID_DATA,message)

measurements_obj = response_obj["data"]

VA_obj = [k for k in measurements_obj if k['type'] == 'VA']

VA_obj_a = [k for k in measurements_obj if k['type'] == 'VA' and k['phases'] == 'A']
VA_obj_b = [k for k in measurements_obj if k['type'] == 'VA'and k['phases'] == 'B']
VA_obj_c = [k for k in measurements_obj if k['type'] == 'VA' and k['phases'] == 'C']

VA_obj_mrid_a = [k['measid'] for k in VA_obj_a]
VA_obj_mrid_b = [k['measid'] for k in VA_obj_b]
VA_obj_mrid_c = [k['measid'] for k in VA_obj_c]

# 5. Create Simulation Subscription Method or Function

Note: Instead chose to query Timeseries Database

In [49]:
def my_timeseries_query(VA_obj_mrid_a, VA_obj_mrid_b, VA_obj_mrid_c):
    topic = "goss.gridappsd.process.request.data.timeseries" # Specify Timeseries API GridAPPS-D topic
    
    # Get time that capacitor diff message was sent
    diff_message_time = int(time.time())-60
    
    # Wait 10 seconds for cap command to execute, new measurement to be created, 
    # then measurement saved to Timeseries Influx DB. This takes a bit of time, so wait to query
    #time.sleep(20)
    

    
    # Query for a particular set of measurments, starting at time cap was commanded
    message = {
        "queryMeasurement":"simulation",
        "queryFilter":{"simulation_id": viz_simulation_id,
                       "startTime": diff_message_time, 
                       "measurement_mrid":VA_obj_mrid_a},
        "responseFormat":"JSON"
    } 

    
    VA_measurement_a = gapps.get_response(topic, message)  
    VA_measurement_obj_a = VA_measurement_a['data']
    


    # Query for a particular set of measurments
    message = {
        "queryMeasurement":"simulation",
        "queryFilter":{"simulation_id": viz_simulation_id,
                       "startTime": diff_message_time, 
                       "measurement_mrid":VA_obj_mrid_b},
        "responseFormat":"JSON"
    } 

    VA_measurement_b = gapps.get_response(topic, message)  
    VA_measurement_obj_b = VA_measurement_b['data']

   
    # Query for a particular set of measurments
    message = {
        "queryMeasurement":"simulation",
        "queryFilter":{"simulation_id": viz_simulation_id,
                       "startTime": diff_message_time, 
                       "measurement_mrid":VA_obj_mrid_c},
        "responseFormat":"JSON"
    } 

    VA_measurement_c = gapps.get_response(topic, message)  
    VA_measurement_obj_c = VA_measurement_c['data']

    VA_meas_mag_a = []
    VA_meas_ang_a = []
    for index in VA_measurement_obj_a:
        magnitude = index['magnitude']
        angle = index['angle']
        VA_meas_mag_a.append(magnitude)
        VA_meas_ang_a.append(angle)

    VA_meas_mag_b = []
    VA_meas_ang_b = []
    for index in VA_measurement_obj_b:
        magnitude = index['magnitude']
        angle = index['angle']
        VA_meas_mag_b.append(magnitude)
        VA_meas_ang_b.append(angle)

    VA_meas_mag_c = []
    VA_meas_ang_c = []
    for index in VA_measurement_obj_c:
        magnitude = index['magnitude']
        angle = index['angle']
        VA_meas_mag_c.append(magnitude)
        VA_meas_ang_c.append(angle)
        
    P_a = VA_meas_mag_a[-1]*np.cos(VA_meas_ang_a[-1]*np.pi/180)
    P_b = VA_meas_mag_b[-1]*np.cos(VA_meas_ang_b[-1]*np.pi/180)
    P_c = VA_meas_mag_c[-1]*np.cos(VA_meas_ang_c[-1]*np.pi/180)
    P = P_a + P_b + P_c

    return P

# 6. Run Optimization, Get Results

[Link to list of control attributes for difference message](Lesson%202.6.%20Simulation%20API%20-%20Publishing%20and%20Subscribing.ipynb#3.2.-Obtain-Equipment-Control-mRIDs)

In [54]:
# Get control attribute from Lesson 2.6, Section 3.2

input_topic = simulation_input_topic(viz_simulation_id)

my_diff_build = DifferenceBuilder(viz_simulation_id)

P_list=[]
P = my_timeseries_query(VA_obj_mrid_a, VA_obj_mrid_b, VA_obj_mrid_c)
P_list.append(P)

# Open all Caps
for k in ShuntComp_id:
    my_diff_build.add_difference(k, "ShuntCompensator.sections", 0, 1) # status change for capacitor given by cap_mrid
    message = my_diff_build.get_message()
    gapps.send(input_topic,message)

time.sleep(60)

P = my_timeseries_query(VA_obj_mrid_a, VA_obj_mrid_b, VA_obj_mrid_c)
P_list.append(P)
    
for k in ShuntComp_id:
    my_diff_build.add_difference(k, "ShuntCompensator.sections", 1, 0) # status change for capacitor given by cap_mrid
    message = my_diff_build.get_message()
    gapps.send(input_topic,message)
    
time.sleep(60)

P = my_timeseries_query(VA_obj_mrid_a, VA_obj_mrid_b, VA_obj_mrid_c)
P_list.append(P)

P_max = max(P_list)
P_min = min(P_list)
print("Worst value is", P_max)
print(".....")
print("Best value is", P_min)

Worst value is 3745407.546634
.....
Best value is 3694657.0859089997
